# App for analyzing GECCO runs from the Galaxy

1. Upload local data or query results of the GECCO from the Galaxy.
2. Identifying Biosynthetic Gene Clusters (BGCs).
3. Visualize BGCs.
4. Compare two samples in respect to each other.

Note: Sending GECCO jobs to Galaxy is part of another separate application.

## Platform dependent part
- Resolve platform setup
- the difference to local imports should be resolved by setting the VRE packages well

In [ ]:
import os
import sys
import logging
import psutil
from IPython import get_ipython

logger = logging.getLogger(name="GECCO galaxy runner")

if 'google.colab' in str(get_ipython()):
    # clone the momics-demos repository to use the utils module from there
    # TODO: eventually utils from momics will be used for that
    try:
        os.system('git clone https://github.com/palec87/momics-demos.git')
        logger.info(f"Repository cloned")
    except OSError as e:
        logger.info(f"An error occurred while cloning the repository: {e}")

    sys.path.insert(0,'/content/momics-demos')

# I do not know how to distinguish between binder and local jupyter server
elif psutil.users()[0].name not in ['davidp', 'david']:  # this is binder 
# elif "zmqshell" in str(get_ipython()):
    logger.info("Binder")
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

    logger.info('Binder will not allow you to upload the ".env" file')
    os.environ["GALAXY_EARTH_URL"] = "https://earth-system.usegalaxy.eu/"
    #########################
    ### INPUT TOKEN HERE ####
    #########################
    os.environ["GALAXY_EARTH_KEY"] = 0
    assert isinstance(os.environ["GALAXY_EARTH_KEY"], str) is True, "token must be a string"

else:
    logger.info("Local server")
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))  # local utils, to be removed in the future

    # downside of this is that all the deps need to be installed in the current (momics-demos) environment
    sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../../../marine-omics')))  # local momics package, to be removed too

from utils import init_setup, get_notebook_environment
init_setup()

# Determine the notebook environment
env = get_notebook_environment()
logger.info(f"Environment: {env}")

In [ ]:
# This needs to be repeated here for the Pannel dashboard to work, WEIRD
# TODO: report as possible bug
import sys
import os
import io

# import numpy as np
import pandas as pd
# import matplotlib.pyplot as plt
# import seaborn as sns
import panel as pn

# Import
import bioblend.galaxy as g  # BioBlend is a Python library, wrapping the functionality of Galaxy and CloudMan APIs
from bioblend.galaxy import GalaxyInstance

# All low level functions are imported from the momics package
import momics.diversity as div
import momics.plotting as pl
from momics.panel_utils import diversity_select_widgets, create_indicators_diversity
from momics.utils import memory_load
from momics.loader import bytes_to_df

## Loading and setup

In [ ]:
if 'google.colab' in str(get_ipython()):
    root_folder = os.path.abspath(os.path.join('/content/momics-demos'))
else:
    root_folder = os.path.abspath(os.path.join('../'))

assets_folder = os.path.join(root_folder, 'assets')

## App Setup

In [ ]:
pn.extension('tabulator')
df_grid = pn.GridBox()
DATASETS = {}


def display_tables_after_upload(datasets):
    logger.info("Displaying tables after upload...")
    tabs = {
        key: pn.widgets.Tabulator(
            data,
        )
        for key, data in datasets.items()
    }
    df_grid.objects = list(tabs.values())

def process_uploaded_tables(file_names, file_data):
    """
    Process the uploaded tables and display them in a tabular format.
    """
    logger.info("Processing uploaded tables...")
    logger.info(f"files: {file_names}")
    DATASETS.clear()
    if file_names is []:
        return
    for i, name in enumerate(file_names):
        DATASETS[name] = bytes_to_df(file_data[i])
    # Display the first table
    display_tables_after_upload(DATASETS)

In [ ]:
pn.extension("tabulator", "mathjax", "filedropper")
ACCENT = "teal"

styles = {
    "box-shadow": "rgba(50, 50, 93, 0.25) 0px 6px 12px -2px, rgba(0, 0, 0, 0.3) 0px 3px 7px -3px",
    "border-radius": "4px",
    "padding": "10px",
}

image = pn.pane.JPG(os.path.join(assets_folder, "figs/logo_gecco.jpeg"),
                    width=100, height=100)

markdown_intro = pn.pane.Markdown(
"""
**If you want to use the Galaxy API:**
- you need to provide your credentials.
- and `job_id` of the job that created the files to get those files.
- For comparison study, you need to provide two job_ids.
"""
)

md_upload = pn.pane.Markdown(
"""
**Upload your files:**
You need to select exactly 3 files with endings:
- `_BGC.tsv`
- `_features.tsv`
- `_genes.tsv`
"""
)

md_get_galaxy = pn.pane.Markdown(
"""
**Get files from Galaxy:**
You need to provide your credentials and `job_id`.
"""
)

upload_local = pn.widgets.FileInput(
    multiple=True,
)

literal_galaxy_url = pn.widgets.TextInput(
    name='Galaxy server URL',
    placeholder='Enter a https server address here...',
)

literal_galaxy_key = pn.widgets.PasswordInput(
    name='Password',
    placeholder='Enter your password here...',
)

button_display_loaded = pn.widgets.Button(
    name="Display loaded files",
    button_type="primary",
    width=200,
)

pn.bind(
    process_uploaded_tables,
    file_names=upload_local.filename,
    file_data = upload_local.value,
    watch=True,
)

button_display_loaded.on_click(
    lambda event: process_uploaded_tables(upload_local.filename, upload_local.value)
)

# tabs = pn.Tabs(
#     ("Table 1", table1),
#     dynamic=True,
#     styles=styles, sizing_mode="stretch_width", height=500, margin=10,
# )


In [ ]:
# upload_local

In [ ]:
# upload_local.value

In [ ]:
# upload_local.filename

In [ ]:
# button_display_loaded

In [ ]:
# df_grid

In [ ]:
# DATASETS

In [ ]:
pn.extension("tabulator",
             "mathjax",
            #  "filedropper",
             )

template = pn.template.FastListTemplate(
    title="DBiosynthetic Gene Cluster Analysis",
    sidebar=[image,
             md_upload, 
             pn.Row(upload_local),
             pn.layout.Divider(margin=(-20, 0, 0, 0)),
             md_get_galaxy, literal_galaxy_url, literal_galaxy_key,
             button_display_loaded,
            #  get_galaxy_files,
            #  pn.layout.Divider(margin=(-20, 0, 0, 0)),
            #  "# Beta diversity", select_table_beta, select_taxon, select_beta_factor,
             ],
    main=[pn.Column(markdown_intro,
                    pn.layout.Divider(margin=(-20, 0, 0, 0)),
                    df_grid,
                    sizing_mode="stretch_both",
                   )],
    main_layout=None,
    accent=ACCENT,
    # meta_refresh="2",
)

template.servable()